### Unified SRM CFD Solver

Solid Rocket Motor(SRM) Simulation using FoamLib/OpenFOAM. Includes nozzle mesh and species modeling. Checks for convergence and stability of the simulation. 
1. Foamlib: https://foamlib.readthedocs.io/en/stable/
2. OpenFOAM: https://www.openfoam.com/

Implementing:
- Axisymmetric bell nozzle
- Compressible reacting flow
- Turbulence (kOmegaSST)
- Burn-rate law
- CFL monitoring
- Residual and physical convergence checks

In [1]:
import os
import shutil
import tempfile
import subprocess
from dataclasses import dataclass
from typing import Dict, Callable

import os 
import shutil 
import tempfile 
import numpy as np 
import matplotlib.pyplot as plt 
from dataclasses import dataclass, field 
from typing import List, Dict, Callable

In [2]:


# =========================
# FoamLib Healthcheck
# =========================

def foamlib_healthcheck() -> bool:
    """
    Verifies:
    1. FoamLib can be imported
    2. OpenFOAM executables are accessible
    3. A minimal case can be generated
    """
    try:
        import foamlib
        from foamlib import FoamCase
    except ImportError as e:
        raise RuntimeError("FoamLib not importable") from e

    # Check OpenFOAM executables
    required_bins = ["blockMesh", "rhoReactingFoam"]
    for b in required_bins:
        if shutil.which(b) is None:
            raise RuntimeError(f"Missing OpenFOAM executable: {b}")

    return True


# =========================
# Geometry & Mesh
# =========================

@dataclass
class BellNozzleGeometry:
    chamber_radius: float
    throat_radius: float
    exit_radius: float
    chamber_length: float
    nozzle_length: float

    def contour(self, n: int = 50):
        """
        Returns (x, r) contour for axisymmetric nozzle
        Rao-style smooth bell (simplified polynomial)
        """
        import numpy as np

        x = np.linspace(0, self.nozzle_length, n)
        r = self.throat_radius + (self.exit_radius - self.throat_radius) * (
            3*(x/self.nozzle_length)**2 - 2*(x/self.nozzle_length)**3
        )
        return x, r




In [3]:
# def generate_blockmesh_dict(geom: BellNozzleGeometry) -> str:
#     """
#     Valid 2D axisymmetric blockMeshDict with single-cell thickness and empty boundary.
#     """
#     L = geom.chamber_length
#     R = geom.chamber_radius
#     dz = 1e-6  # tiny thickness for 2D

#     return f"""
# FoamFile
# {{
#     version     2.0;
#     format      ascii;
#     class       dictionary;
#     location    "system";
#     object      blockMeshDict;
# }}

# convertToMeters 1;

# vertices
# (
#     (0 0 0)        // 0
#     ({L} 0 0)      // 1
#     ({L} {R} 0)    // 2
#     (0 {R} 0)      // 3

#     (0 0 {dz})        // 4
#     ({L} 0 {dz})      // 5
#     ({L} {R} {dz})    // 6
#     (0 {R} {dz})      // 7
# );

# blocks
# (
#     hex (0 1 2 3 4 5 6 7) (200 80 1) simpleGrading (1 1 1)
# );

# edges ();

# boundary
# (
#     inlet
#     {{
#         type wall;
#         faces ((0 3 7 4));
#     }}

#     outlet
#     {{
#         type patch;
#         faces ((1 2 6 5));
#     }}

#     wall
#     {{
#         type wall;
#         faces ((3 2 6 7));
#     }}

#     axis
#     {{
#         type empty;
#         faces ((0 1 5 4));
#     }}
# );

# mergePatchPairs ();
# """


In [4]:
def generate_blockmesh_dict(geom: BellNozzleGeometry) -> str:
    import numpy as np

    nx = 200
    nr = 80
    nz = 1
    thickness = 0.01
    angle_deg = 2.5
    offset = thickness * np.tan(np.radians(angle_deg))

    # vertices
    v0 = (0, 0, 0)
    v1 = (geom.chamber_length, 0, 0)
    v2 = (geom.chamber_length, geom.chamber_radius, 0)
    v3 = (0, geom.chamber_radius, 0)

    v4 = (0, 0 + offset, thickness)
    v5 = (geom.chamber_length, 0 + offset, thickness)
    v6 = (geom.chamber_length, geom.chamber_radius, thickness)
    v7 = (0, geom.chamber_radius, thickness)

    return f"""FoamFile
{{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}}

convertToMeters 1;

vertices
(
    ({v0[0]} {v0[1]} {v0[2]})
    ({v1[0]} {v1[1]} {v1[2]})
    ({v2[0]} {v2[1]} {v2[2]})
    ({v3[0]} {v3[1]} {v3[2]})

    ({v4[0]} {v4[1]} {v4[2]})
    ({v5[0]} {v5[1]} {v5[2]})
    ({v6[0]} {v6[1]} {v6[2]})
    ({v7[0]} {v7[1]} {v7[2]})
);

blocks
(
    hex (0 1 2 3 4 5 6 7) ({nx} {nr} {nz}) simpleGrading (1 1 1)
);

edges ();

boundary
(
    inlet
    {{
        type patch;
        faces ((0 3 7 4));
    }}

    outlet
    {{
        type patch;
        faces ((1 2 6 5));
    }}

    wall
    {{
        type wall;
        faces ((3 2 6 7));
    }}

    front
    {{
        type empty;
        faces ((0 1 5 4));
    }}

    back
    {{
        type empty;
        faces ((3 2 6 7));
    }}
);

mergePatchPairs
(
);
"""


In [5]:
fvSchemes_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "system";
    object      fvSchemes;
}

ddtSchemes
{
    default         Euler;
}

gradSchemes
{
    default         Gauss linear;
}

divSchemes
{
    default         none;
    div(phi,U)      Gauss upwind;
    div(phi,T)      Gauss upwind;
    div(phi,Ydefault) Gauss upwind;
}

laplacianSchemes
{
    default         Gauss linear corrected;
}

interpolationSchemes
{
    default         linear;
}

snGradSchemes
{
    default         corrected;
}
"""


In [6]:
fvSolution_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "system";
    object      fvSolution;
}

solvers
{
    p
    {
        solver          PCG;
        preconditioner  DIC;
        tolerance       1e-6;
        relTol          0;
    }

    U
    {
        solver          PBiCG;
        preconditioner  DILU;
        tolerance       1e-6;
        relTol          0.1;
    }

    T
    {
        solver          PBiCG;
        preconditioner  DILU;
        tolerance       1e-6;
        relTol          0.1;
    }

    Ydefault
    {
        solver          PBiCG;
        preconditioner  DILU;
        tolerance       1e-6;
        relTol          0.1;
    }
}

SIMPLE
{
    nNonOrthogonalCorrectors 0;
}
"""

In [7]:
controlDict_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "system";
    object      controlDict;
}

application     rhoReactingFoam;

startFrom       startTime;

startTime       0;

stopAt          endTime;

endTime         0.01;

deltaT          1e-6;

writeControl    timeStep;

writeInterval   1;

purgeWrite      0;

writeFormat     ascii;

writePrecision  6;

writeCompression off;

timeFormat      general;

timePrecision   6;

runTimeModifiable true;
"""

In [8]:
thermophysicalProperties_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant";
    object      thermophysicalProperties;
}

thermoType
{
    type            heRhoThermo;
    mixture         homogeneousMixture;
    transport       const;
    thermo          hConst;
    equationOfState perfectGas;
    specie          specie;
    energy          sensibleEnthalpy;
}

inertSpecie gas; 

reactants
{
    specie
    {
        nMoles          1;
        molWeight       28.9; // Adjust for your specific gas
    }
    thermodynamics
    {
        Cp              1007;
        Hf              0;
    }
    transport
    {
        mu              1.8e-05;
        Pr              0.7;
    }
}

products
{
    specie
    {
        nMoles          1;
        molWeight       28.9; 
    }
    thermodynamics
    {
        Cp              1007;
        Hf              0;
    }
    transport
    {
        mu              1.8e-05;
        Pr              0.7;
    }
}
"""



In [9]:
specie_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant/thermophysicalProperties";
    object      specie;
}

specie
(
    gas
    {
        molWeight   28.96; 
    }
)
"""


In [10]:
mixture_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant/thermophysicalProperties";
    object      mixture;
}

mixture
{
    specie      (gas);
    fraction    (1.0);
}
"""


In [11]:
transport_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant/thermophysicalProperties";
    object      transport;
}

transportModel  const;
mu              1.8e-05;
Pr              0.7;
}
"""

In [12]:
thermo_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant/thermophysicalProperties";
    object      thermo;
}

Cp              1005;
Hf              0;
}
"""

In [13]:
turbulenceProperties_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant";
    object      turbulenceProperties;
}

simulationType  RAS;

RAS
{
    RASModel        kOmegaSST;
    turbulence      on;
    printCoeffs     on;
}
"""


In [14]:
# -------------------------
# Initial pressure field
# -------------------------
p_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "0";
    object      p;
}

dimensions      [1 -1 -2 0 0 0 0];

internalField   uniform 101325;

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform 101325;
    }
    outlet
    {
        type            zeroGradient;
    }
    wall
    {
        type            zeroGradient;
    }
    front
    {
        type            empty;
    }
    back
    {
        type            empty;
    }
}
"""

# -------------------------
# Initial velocity field
# -------------------------
U_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       volVectorField;
    location    "0";
    object      U;
}

dimensions      [0 1 -1 0 0 0 0];

internalField   uniform (0 0 0);

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform (0 0 0);
    }
    outlet
    {
        type            zeroGradient;
    }
    wall
    {
        type            noSlip;
    }
    front
    {
        type            empty;
    }
    back
    {
        type            empty;
    }
}
"""

# -------------------------
# Initial temperature field
# -------------------------
T_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "0";
    object      T;
}

dimensions      [0 0 0 1 0 0 0];

internalField   uniform 300;

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform 300;
    }
    outlet
    {
        type            zeroGradient;
    }
    wall
    {
        type            zeroGradient;
    }
    front
    {
        type            empty;
    }
    back
    {
        type            empty;
    }
}
"""

# -------------------------
# Initial species mass fraction field
# -------------------------
Yi_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "0";
    object      Yi;
}

dimensions      [0 0 0 0 0 0 0];

internalField   uniform 0.21;

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform 0.21;
    }
    outlet
    {
        type            zeroGradient;
    }
    wall
    {
        type            zeroGradient;
    }
    axis
    {
        type            empty;
    }
}
"""

In [15]:
Ydefault_text = """FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "0";
    object      Ydefault;
}

dimensions      [0 0 0 0 0 0 0];

internalField   uniform 1.0;

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform 1.0;
    }

    outlet
    {
        type            zeroGradient;
    }

    wall
    {
        type            zeroGradient;
    }

    front
    {
        type            empty;
    }

    back
    {
        type            empty;
    }
}
"""

In [16]:
# Ydefault_text = (
# "FoamFile\n"
# "{\n"
# "    version     2.0;\n"
# "    format      ascii;\n"
# "    class       volScalarField;\n"
# "    location    \"0\";\n"
# "    object      Ydefault;\n"
# "}\n"
# "\n"
# "dimensions      [0 0 0 0 0 0 0];\n"
# "\n"
# "internalField   uniform 1.0;\n"
# "\n"
# "boundaryField\n"
# "{\n"
# "    inlet\n"
# "    {\n"
# "        type            fixedValue;\n"
# "        value           uniform 1.0;\n"
# "    }\n"
# "\n"
# "    outlet\n"
# "    {\n"
# "        type            zeroGradient;\n"
# "    }\n"
# "\n"
# "    walls\n"
# "    {\n"
# "        type            zeroGradient;\n"
# "    }\n"
# "\n"
# "    front\n"
# "    {\n"
# "        type            wedge;\n"
# "    }\n"
# "\n"
# "    back\n"
# "    {\n"
# "        type            wedge;\n"
# "    }\n"
# "}\n"
# )

In [17]:
# =========================
# FoamLib Healthcheck
# =========================
def foamlib_healthcheck() -> bool:
    """Check FoamLib and OpenFOAM environment."""
    try:
        import foamlib
        from foamlib import FoamCase
    except ImportError as e:
        raise RuntimeError("FoamLib not importable") from e

    required_bins = ["blockMesh", "rhoReactingFoam"]
    for b in required_bins:
        if shutil.which(b) is None:
            raise RuntimeError(f"Missing OpenFOAM executable: {b}")

    return True

In [18]:
# =========================
# Geometry & Mesh
# =========================
@dataclass
class BellNozzleGeometry:
    chamber_radius: float
    throat_radius: float
    exit_radius: float
    chamber_length: float
    nozzle_length: float

    def contour(self, n: int = 50):
        """Return (x, r) nozzle contour for axisymmetric 2D."""
        x = np.linspace(0, self.nozzle_length, n)
        r = self.throat_radius + (self.exit_radius - self.throat_radius) * (
            3*(x/self.nozzle_length)**2 - 2*(x/self.nozzle_length)**3
        )
        return x, r

In [19]:
# =========================
# Physics Models
# =========================
@dataclass
class CombustionModel:
    burn_rate_coeff: float
    pressure_exponent: float

    def burn_rate(self, p: float) -> float:
        """Burn rate law: r = a * p^n"""
        return self.burn_rate_coeff * (p ** self.pressure_exponent)


@dataclass
class TurbulenceModel:
    model: str = "kOmegaSST"

    def fv_turbulence_dict(self) -> Dict:
        return {
            "simulationType": "RAS",
            "RAS": {
                "RASModel": self.model,
                "turbulence": "on",
                "printCoeffs": "on"
            }
        }


# =========================
# Stability & Convergence
# =========================
@dataclass
class CFLMonitor:
    max_cfl_allowed: float = 0.5
    history: List[float] = field(default_factory=list)

    def update(self, velocity, dt, dx):
        cfl = np.max(np.abs(velocity)) * dt / dx
        self.history.append(cfl)
        if cfl > self.max_cfl_allowed:
            raise RuntimeError(f"CFL violation: {cfl:.3f} > {self.max_cfl_allowed}")
        return cfl


@dataclass
class ResidualMonitor:
    tolerances: Dict[str, float]
    history: Dict[str, List[float]] = field(default_factory=dict)

    def check(self, residuals: Dict[str, float]) -> bool:
        for field, res in residuals.items():
            if field not in self.history:
                self.history[field] = []
            self.history[field].append(res)
        for field, res in residuals.items():
            if res > self.tolerances.get(field, 1e-6):
                return False
        return True

    def plot(self):
        plt.figure(figsize=(8, 5))
        for field, values in self.history.items():
            plt.semilogy(values, label=field)
            plt.axhline(self.tolerances.get(field, 1e-6), color='r', linestyle='--', label=f'{field} tolerance')
        plt.xlabel("Timestep")
        plt.ylabel("Residual (log scale)")
        plt.title("SRM Residuals Over Time")
        plt.legend()
        plt.grid(True, which="both", linestyle="--", linewidth=0.5)
        plt.tight_layout()
        plt.show()


@dataclass
class PhysicalConvergenceMonitor:
    tolerance: float = 1e-3
    window: int = 5
    pressure_history: List[float] = field(default_factory=list)
    mdot_history: List[float] = field(default_factory=list)

    def update(self, pc, mdot) -> bool:
        self.pressure_history.append(pc)
        self.mdot_history.append(mdot)
        if len(self.pressure_history) < self.window:
            return False
        dp = abs(self.pressure_history[-1] - self.pressure_history[-self.window]) / pc
        dm = abs(self.mdot_history[-1] - self.mdot_history[-self.window]) / mdot
        return (dp < self.tolerance) and (dm < self.tolerance)




In [20]:
# =========================
# SRM Solver Wrapper
# =========================
class SRMSolver:
    """High-level FoamLib SRM wrapper (axisymmetric, reacting, compressible)."""

    def __init__(self, geometry, combustion, turbulence, case_dir):
        from foamlib import FoamCase
        self.case = FoamCase(case_dir)
        self.case_dir = case_dir
        self.geometry = geometry
        self.combustion = combustion
        self.turbulence = turbulence

        # Monitors
        self.dx = 0.005
        self.dt = 1e-6
        self.cfl_monitor = CFLMonitor(max_cfl_allowed=0.5)
        self.residual_monitor = ResidualMonitor({
            "p": 1e-4, "U": 1e-5, "T": 1e-5, "Yi": 1e-6
        })
        self.physical_monitor = PhysicalConvergenceMonitor()

    def setup_mesh(self):
        """
        Create the system folder, write a corrected blockMeshDict for
        axisymmetric 2D wedge geometry, run blockMesh, and validate.
        """

        case_dir = self.case_dir
        system_dir = os.path.join(case_dir, "system")
        os.makedirs(system_dir, exist_ok=True)

        # Write the corrected blockMeshDict
        blockMeshDict_path = os.path.join(system_dir, "blockMeshDict")
        with open(blockMeshDict_path, "w") as f:
            f.write(generate_blockmesh_dict(self.geometry))

        print(f"blockMeshDict written to {blockMeshDict_path}")

        # Run blockMesh inside the case directory
        try:
            self.case.run("blockMesh")
        except Exception as e:
            raise RuntimeError(
                f"blockMesh failed. Check the blockMeshDict at {blockMeshDict_path}"
            ) from e

        # Validate polyMesh creation
        polyMesh_points = os.path.join(case_dir, "constant", "polyMesh", "points")
        if not os.path.exists(polyMesh_points):
            raise RuntimeError(
                "blockMesh failed: constant/polyMesh/points was not created. "
                "Check wedge definition and mesh dimensions."
            )

        print("blockMesh completed successfully, polyMesh directory created.")

        
        
    def setup_physics(self, case_dir, system_dir):
        
        system_dir = os.path.join(self.case_dir, "system")
        with open(os.path.join(system_dir, "controlDict"), "w") as f:
            f.write(controlDict_text)
            
#         with open(os.path.join(system_dir, "turbulenceProperties"), "w") as f:
#             f.write(turbulenceProperties_text)

#        

        # Write fvSchemes
        with open(os.path.join(system_dir, "fvSchemes"), "w") as f:
            f.write(fvSchemes_text)  # paste the text above as a multi-line string

        # Write fvSolution
        with open(os.path.join(system_dir, "fvSolution"), "w") as f:
            f.write(fvSolution_text)  # paste the text above as a multi-line string
            
    def setup_thermo(self, thermo_dir, constant_dir):

#         with open(os.path.join(thermo_dir, "specie"), "w") as f:
#             f.write(specie_text)

#         with open(os.path.join(thermo_dir, "mixture"), "w") as f:
#             f.write(mixture_text)

#         with open(os.path.join(thermo_dir, "transport"), "w") as f:
#             f.write(transport_text)
            
#         with open(os.path.join(thermo_dir, "thermo"), "w") as f:
#             f.write(thermo_text)
            
        with open(os.path.join(constant_dir, "thermophysicalProperties"), "w") as f:
            f.write(thermophysicalProperties_text)

#         with open(os.path.join(constant_dir, "turbulenceProperties"), "w") as f:
#             f.write(turbulenceProperties_text)



    def run(self):
        """Run solver with mock residual monitoring."""
        self.case.run("rhoReactingFoam")

        # Placeholder for demo: simulate 50 timesteps with mock data
        for step in range(50):
            velocity = np.array([300.0 + np.random.randn()*2])
            residuals = {
                "p": 1e-5 + np.random.rand()*1e-6,
                "U": 1e-6 + np.random.rand()*1e-7,
                "T": 1e-6 + np.random.rand()*1e-7,
                "Yi": 1e-7 + np.random.rand()*1e-8
            }
            pc = 6e6 + np.random.randn()*1e3
            mdot = 12.0 + np.random.randn()*0.01

            # Update monitors
            self.cfl_monitor.update(velocity, self.dt, self.dx)
            self.residual_monitor.check(residuals)
            self.physical_monitor.update(pc, mdot)

    def plot_residuals(self):
        self.residual_monitor.plot()


In [21]:
def build_solver(case_dir, system_dir, constant_dir):
    geom = BellNozzleGeometry(0.05, 0.02, 0.08, 0.2, 0.3)
    combustion = CombustionModel(5e-5, 0.35)
    turb = TurbulenceModel()

    solver = SRMSolver(geom, combustion, turb, case_dir)
    
    solver.setup_thermo(constant_dir, constant_dir)
    solver.setup_physics(case_dir, system_dir)   # writes system/constant files
    solver.setup_mesh()      # writes blockMeshDict + runs blockMesh

    return solver


In [22]:
# =========================
# Entry point
# =========================
if __name__ == "__main__":
    
    foamlib_healthcheck()
    
    case_dir = "./srm_run"
    
    system_dir = os.path.join(case_dir, "system")
    constant_dir = os.path.join(case_dir, "constant")
    thermo_dir = os.path.join(case_dir, "constant")
    zero_dir = os.path.join(case_dir, "0")
    
    if os.path.exists(case_dir):
        shutil.rmtree(case_dir)
    os.makedirs(case_dir, exist_ok=True)
    
    if os.path.exists(system_dir):
        shutil.rmtree(system_dir)
    os.makedirs(system_dir, exist_ok=True)
    
    if os.path.exists(constant_dir):
        shutil.rmtree(constant_dir)
    os.makedirs(constant_dir, exist_ok=True)
    
#     with open(os.path.join(constant_dir, "thermophysicalProperties"), "w") as f:
#             f.write(thermophysicalProperties_text)
    
    if os.path.exists(zero_dir):
        shutil.rmtree(zero_dir)
    os.makedirs(zero_dir, exist_ok=True)
    
    fields = {"p": p_text, "U": U_text, "T": T_text, "Yi": Yi_text}
    for field_name, text in fields.items():
        with open(os.path.join(zero_dir, field_name), "w", newline="\n") as f:
            f.write(text)
            
    cleaned_text = Ydefault_text.replace('\xa0', ' ')

    with open(os.path.join(zero_dir, "Ydefault"), "w", encoding="ascii") as f:
        f.write(cleaned_text.strip() + "\n")

    print("File 0/Ydefault created and cleaned of non-ASCII characters.")
            

    print("0/ folder created with p, U, T, Yi")
    
    if os.path.exists(thermo_dir):
        shutil.rmtree(thermo_dir)
    os.makedirs(thermo_dir, exist_ok=True)
    
    
    
    
    print("thermophysicalProperties folder created with specie, mixture, transport, thermo")


    solver = build_solver(case_dir, system_dir, constant_dir)
    
    
    print(f"\n Solver generated --- \n")
#     solver.run()
#     solver.plot_residuals()
#     print("Axisymmetric SRM simulation complete. Residual plot generated ✔")


/Users/rckyi/miniconda3/lib/python3.10/site-packages/rich/live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/rckyi/miniconda3/lib/python3.10/site-packages/rich/live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

File 0/Ydefault created and cleaned of non-ASCII characters.
0/ folder created with p, U, T, Yi
thermophysicalProperties folder created with specie, mixture, transport, thermo
blockMeshDict written to ./srm_run/system/blockMeshDict


blockMesh completed successfully, polyMesh directory created.

 Solver generated --- 



In [23]:
# solver = build_solver(case_dir, system_dir)
print(f"\n Solver generated --- \n")
solver.run()

/Users/rckyi/miniconda3/lib/python3.10/site-packages/rich/live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


 Solver generated --- 



CalledProcessError: Command '['bash', '-c', 'rhoReactingFoam']' returned non-zero exit status 1.


--> FOAM FATAL IO ERROR: (openfoam-2506)
Inert specie gas not found in available species 1(b)

file: constant/thermophysicalProperties at line 12 to 57.

    From rhoReactingFoam
    in file ./createFields.H at line 14.

FOAM exiting

